# TGP Stage 2 analysis

This notebook performs an interactive analysis of one of the **Stage 2** datasets.
It starts by loading the raw data and proceeds with pre-processing and analysis steps, where each step is visualized.

In [ ]:
from __future__ import annotations

from pathlib import Path
import matplotlib.pyplot as plt

import xarray as xr
import tgp

tgp.plot.set_mpl_rc()

folder = Path("..") / "data"

if not folder.exists():
    folder = Path("/usr/tgp/data")

print("Using tgp code version:", tgp.__version__)

# Analysis on experimental data

First, we select the dataset we want to analyze. The dictionary below contains all of the datasets that are provided.

Change the `selected = "A1"` parameter to e.g., `selected = "B"` to analyze the data of device B instead of device A1.

In [ ]:
from typing import NamedTuple


class Parameters(NamedTuple):
    lockin_1: str
    lockin_2: str
    fridge: str
    drop_indices: list[int] | tuple[list[int], list[int]]
    max_bias_index: int | tuple[int, int]
    selected_cutter: int
    selected_clusters: list[int]
    gap_threshold_high: float = 70e-3
    phase_shift_left: float = 0.0
    phase_shift_right: float = 0.0


parameters_dict = {
    "A1": Parameters(  # ✅ passed TGP
        lockin_1="lockin_1",
        lockin_2="lockin_2",
        fridge="deviceA1.yaml",
        drop_indices=[0, 1, 2],
        max_bias_index=-2,
        selected_cutter=0,
        selected_clusters=[1],
        phase_shift_left=-3.3,
        phase_shift_right=-5.57,
    ),
    "A2": Parameters(  # ✅ passed TGP
        lockin_1="mfli_5510",
        lockin_2="mfli_5602",
        fridge="deviceA2.yaml",
        drop_indices=[0, 1, 2],
        max_bias_index=-2,
        selected_cutter=1,
        selected_clusters=[1],
    ),
    "A3": Parameters(  # ✅ passed TGP
        lockin_1="mfli_5510",
        lockin_2="mfli_5602",
        fridge="deviceA2.yaml",
        drop_indices=[],
        max_bias_index=-1,
        selected_cutter=1,
        selected_clusters=[1],
    ),
    "B": Parameters(  # ✅ passed TGP
        lockin_1="lockin_1",
        lockin_2="lockin_2",
        fridge="deviceB.yaml",
        drop_indices=[0, 1, 2, 3],
        max_bias_index=-4,
        selected_cutter=1,
        selected_clusters=[1],
        gap_threshold_high=100e-3,
        phase_shift_left=-3.78,
        phase_shift_right=-6.89,
    ),
    "C": Parameters(  # ✅ passed TGP
        lockin_1="mfli_5583",
        lockin_2="mfli_5591",
        fridge="deviceC.yaml",
        drop_indices=([0, 1, 2, 3, 4, 5], []),
        max_bias_index=(-4, -1),
        selected_cutter=0,
        selected_clusters=[1],
    ),
    "D": Parameters(  # ✅ passed TGP
        lockin_1="mfli_4909",
        lockin_2="mfli_5654",
        fridge="deviceD.yaml",
        drop_indices=[0, 1],
        max_bias_index=-4,
        selected_cutter=0,
        selected_clusters=[1, 2],
    ),
    "E": Parameters(  # ❌ did not pass TGP
        lockin_1="mfli_5591",
        lockin_2="mfli_5583",
        fridge="deviceE.yaml",
        drop_indices=[0],
        max_bias_index=-1,
        selected_cutter=1,
        selected_clusters=[],
    ),
}

# Select the dataset we want
selected = "A1"
p = parameters_dict[selected]
fname_left = f"device{selected}_stage2_left.nc"
fname_right = f"device{selected}_stage2_right.nc"

We load the raw data and prepare it for the `tgp` code's analysis.
We call the `prepare` function on the loaded raw datasets to validate that it contains the correct dimensions and variables, it also performs renames (if necessary) to account for different naming schemes in different fridges.

In [ ]:
ds_left = xr.load_dataset(folder / "experimental" / fname_left, engine="h5netcdf")
ds_right = xr.load_dataset(folder / "experimental" / fname_right, engine="h5netcdf")
ds_left = tgp.prepare.prepare(ds_left)
ds_right = tgp.prepare.prepare(ds_right)

In [ ]:
if selected == "D":  # Device D is for a single cutter pair only
    # the code expects this dimension so we add it.
    ds_left = ds_left.expand_dims(cutter_pair_index=[0])
    ds_right = ds_right.expand_dims(cutter_pair_index=[0])

We can view the renamed raw data with the plot below.

In [ ]:
# View the data (before corrections)

print(f"Choose V={ds_left.V.values}")
print(f"Choose cutter_pair_index={ds_left.cutter_pair_index.values}")

tgp.plot.two.plot_data(
    ds_left,
    ds_right,
    V=ds_left.V.median(),
    cutter_pair_index=p.selected_cutter,
)
plt.show()

We apply corrections for the circuit effects in three-terminal electrical transport measurements arising from finite line impedances (as reported in [arXiv:2104.02671](https://arxiv.org/abs/2104.02671)).
The `correct_frequencies` functions corrects the spurious voltage divider effects.
Additionally, the `correct_bias` function ensures that zero-bias is at the correct point.
This is required because zero-bias in the experiment might be slightly offset.

In [ ]:
# Apply corrections
from tgp.frequency_correction import correct_frequencies

ds_left, ds_right = correct_frequencies(
    ds_left,
    ds_right,
    lockin_left=p.lockin_1,  # key in the snapshot
    lockin_right=p.lockin_2,
    fridge_parameters=folder / "fridge" / p.fridge,
    phase_shift_left=p.phase_shift_left,
    phase_shift_right=p.phase_shift_right,

)

ds_left, ds_right = tgp.prepare.correct_bias(
    ds_left,
    ds_right,
    drop_indices=p.drop_indices,
    max_bias_index=p.max_bias_index,
    norm=1e3,
    method="manual",
)

We can view the conductance data again with the plot we used above.

In [ ]:
# View the data (after corrections)

print(f"Choose V={ds_left.V.values}")
print(f"Choose cutter_pair_index={ds_left.cutter_pair_index.values}")

tgp.plot.two.plot_data(
    ds_left,
    ds_right,
    V=ds_left.V.median(),  # choose the voltage here
    cutter_pair_index=p.selected_cutter,
)
plt.show()

We can now perform the first analysis step, which is to extract the value of the gap.
This algorithm is based on thresholding the filtered anti-symmetrized part of the nonlocal conductance.

In [ ]:
# Perform the gap extration, optionally tweak the parameters by passing them
ds_left, ds_right = tgp.two.extract_gap(ds_left, ds_right)

We can see the results of the gap extraction using the plot below.

In [ ]:
# Plot the results of the gap extration

print(f"Choose V={ds_left.V.values}")
print(f"Choose cutter_pair_index={ds_left.cutter_pair_index.values}")

V = ds_left.V.median()

tgp.plot.two.plot_gap_extraction(ds_left, ds_right, p.selected_cutter, V)
tgp.plot.two.plot_extracted_gap(ds_left, ds_right, p.selected_cutter)
plt.show()

In addition to learning in which region the phase diagram is gapped, we need to know where in `(B, V)` space zero-bias peaks (ZPB)s occur.
The `zbp_dataset_derivative` function returns a dataset with a combined left and right gap and ZBPs, without the `left_bias` or `right_bias` dimension.

In [ ]:
zbp_ds = tgp.two.zbp_dataset_derivative(
    ds_left, ds_right, zbp_probability_threshold=0.6
)

To go from a gap array with numerical values to a boolean array where in `(B, V)` space the spectrum is gapped, we apply an upper and lower threshold.

We can set them with the following code.

In [ ]:
tgp.two.set_gap_threshold(
    zbp_ds,
    threshold_low=10e-3,
    threshold_high=p.gap_threshold_high,
)

tgp.plot.zbp.plot_gapped_zbp(zbp_ds, p.selected_cutter)
plt.show()

Finally, we cluster the gapped ZBP array and score the clusters with the `cluster_and_score` function.
This action is performed on every `cutter_pair_value` slice.

In [ ]:
# Perform analysis on all cutter_pair_index values
zbp_ds_done = tgp.two.cluster_and_score(zbp_ds, min_cluster_size=7)

We can plot the results of the analysis with the plot below.

We can see the probabilities of the ZPBs on the left and right, and their joint probabilities (top row).
Below we see the gap (d), the gapped ZPBs (e), and the clusters of the gapped ZBPs (f).
Then in the lowest row, we plot the region of interest (ROI) and the gap inside that region.

In [ ]:
print(f"Choose one of cutter_pair_index={zbp_ds_done.cutter_pair_index.values}")
print(f"Choose one of zbp_cluster_number={zbp_ds_done.zbp_cluster_number.values}")

if "cutter_pair_index" in zbp_ds_done.dims:
    sel = zbp_ds_done.sel(cutter_pair_index=p.selected_cutter)
else:
    sel = zbp_ds_done
tgp.plot.zbp.plot_probability_and_clusters(sel)
if p.selected_clusters:
    tgp.plot.zbp.plot_region_of_interest_2(sel, p.selected_clusters[0])
plt.show()

## Manually select cutter value

The `cluster_and_score` function used above performs the entire analysis on a per `cutter_pair_index` basis and joins the results into an array with the `cutter_pair_index` dimension.
Alternatively, we can select a single cutter value and use the analysis and plotting functions directly.

<div class="alert alert-block alert-info">
The functions below assume a single cutter value.
</div>

In [ ]:
zbp_ds_sel = (
    zbp_ds.isel(cutter_pair_index=p.selected_cutter)
    if "cutter_pair_index" in zbp_ds.dims
    else zbp_ds
)

In [ ]:
tgp.plot.zbp.plot_left_right_zbp_probability(zbp_ds)
plt.show()

In [ ]:
tgp.plot.zbp.plot_joined_probability(zbp_ds)
plt.show()

In [ ]:
tgp.plot.zbp.plot_clusters_zbp(zbp_ds_sel)
plt.show()

In [ ]:
tgp.plot.zbp.plot_probability_and_clusters(zbp_ds_sel)
plt.show()

In [ ]:
if p.selected_clusters:
    tgp.plot.zbp.plot_region_of_interest_2(zbp_ds_sel, zbp_cluster_number=p.selected_clusters[0])
    plt.show()

## Paper plots of the selected cluster

By selecting a single `cutter_pair_index` we can produce the same plot as in the paper.

In [ ]:
import tgp.plot.paper

In [ ]:
tgp.plot.paper.plot_stage2_diagram(
    zbp_ds,
    cutter_value=p.selected_cutter,
    zbp_cluster_numbers=p.selected_clusters,
)
plt.show()

One has to select same `selected_plunger` parameter as in the paper figures notebook to reproduce the same plot. We choose the plunger to be in the center of the selected cluster here.

In [ ]:
if "cutter_pair_index" not in zbp_ds.dims:
    zbp_ds = zbp_ds.expand_dims(cutter_pair_index=[0])

if p.selected_clusters:
    V = zbp_ds.sel(
        zbp_cluster_number=p.selected_clusters[0], cutter_pair_index=p.selected_cutter
    ).cluster_V_center.item()
else:
    raise RuntimeError("Please manually set the V value.")

tgp.plot.paper.plot_conductance_waterfall(
    zbp_ds_sel,
    ds_left,
    ds_right,
    selected_cutter=p.selected_cutter,
    selected_plunger=V,
    zbp_cluster_numbers=p.selected_clusters,
    bias_max=0.06,
    labels="abcd",
)
plt.show()